In [ ]:
import pandas as pd
import os

In [ ]:
# load all sheets from drive
# sheet ID comes from URL
sheet_ids = [
            '1Pxs2JngjAQ_iqTtVuzy0xGcyPP7K7F-rumOiN3DcQJI', 
            '1Ktl2TXZPVme2NgTTt-_MivE7X-sTRnrZYHNj6LTzb9I', 
            '1y5Zuh-v-LyRmxurUzpgcfHHdh2eiybRgGiD5K6bYLT8', 
            '1teYR0T8bsXBTTcb1cf00UNFXDgXzmOCh-W2dl7I_fFY',
            '1p0ErgEP94Y-GhG9K550wsTU6E0SNP8aaTNa0rtKD9pY'
            ] 
sheet_names = [
                'Curso_Basico_Sentence_Pairs', 
                'PS_jw_website', 
                'Warao-Aribu-Learning-Guide-Data',
                'Trab_Warao_Completo',
                'Bible_verse_pairs'
                ]

df_lst = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

for i in range(len(sheet_ids)):
    sheet_id = sheet_ids[i]
    sheet_name = sheet_names[i]
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    df_lst[i] = pd.read_csv(url)

In [ ]:
df_curso_basico, df_ps_jw_website, df_warao_aribu_guide, df_trab_warao_completo, df_bible_verses = df_lst[0], df_lst[1], df_lst[2], df_lst[3], df_lst[4]

## Display Data

In [ ]:
print(f"Number of examples from {sheet_names[0]}: {df_curso_basico.shape[0]}")
display(df_curso_basico.head(10))

In [ ]:
print(f"Number of examples from {sheet_names[1]}: {df_ps_jw_website.shape[0]}")
display(df_ps_jw_website.head(10))

In [ ]:
print(f"Number of examples from {sheet_names[2]}: {df_warao_aribu_guide.shape[0]}")
display(df_warao_aribu_guide.head(10))

In [ ]:
print(f"Number of examples from {sheet_names[3]}: {df_trab_warao_completo.shape[0]}")
display(df_trab_warao_completo.head(10))

In [ ]:
print(f"Number of examples from {sheet_names[4]}: {df_bible_verses.shape[0]}")
display(df_bible_verses.head(10))

## Clean up data

### df_curso_basico

In [ ]:
# combine alternate translations into a single column
expanded_rows = []

for idx, row in df_curso_basico.iterrows():
    # add original row with spanish_sentence_1
    if pd.notna(row['spanish_sentence_1']):
        expanded_rows.append({
            'warao_sentence': row['warao_sentence'],
            'spanish_sentence': row['spanish_sentence_1']
        })
    
    # check for alternate translations in columns spanish_sentence_2-4
    for col in ['spanish_sentence_2', 'spanish_sentence_3', 'spanish_sentence_4']:
        if pd.notna(row[col]):
            expanded_rows.append({
                'warao_sentence': row['warao_sentence'],
                'spanish_sentence': row[col]
            })

# new dataframe
df_curso_basico_expanded = pd.DataFrame(expanded_rows)

print(f"Original number of examples: {len(df_curso_basico)}")
print(f"Expanded number of examples: {len(df_curso_basico_expanded)}")
display(df_curso_basico_expanded.head(20))


### df_warao_aribu

In [ ]:
# if isupper == True, change warao_sentence to lowercase with the first letter capitalized
# Only apply transformation to rows where the string is fully uppercase
mask = df_warao_aribu_guide['warao_sentence'].str.isupper().fillna(False)
df_warao_aribu_guide.loc[mask, 'warao_sentence'] = df_warao_aribu_guide.loc[mask, 'warao_sentence'].str.lower().str.capitalize()

display(df_warao_aribu_guide[2:3])
display(df_warao_aribu_guide[83:94])

### df_trab_warao_completo

In [ ]:
# change certain text that is in all caps text to lowercase with the first letter capitalized
df_trab_warao_completo['warao_sentence'] = df_trab_warao_completo['warao_sentence'].str.lower().str.capitalize()
df_trab_warao_completo = df_trab_warao_completo[['spanish_sentence', 'warao_sentence']]

display(df_trab_warao_completo[100:108])


In [ ]:
duplicates_curso_basico = df_curso_basico_expanded[df_curso_basico_expanded.duplicated(keep=False)]
duplicates_warao_aribu = df_warao_aribu_guide[df_warao_aribu_guide.duplicated(keep=False)]
duplicates_ps_jw = df_ps_jw_website[df_ps_jw_website.duplicated(keep=False)]
duplicates_trab = df_trab_warao_completo[df_trab_warao_completo.duplicated(keep=False)]
duplicates_bible = df_bible_verses[df_bible_verses.duplicated(keep=False)]

print(f"Number of duplicate examples in df_curso_basico_expanded: {len(duplicates_curso_basico)}")
print(f"Number of duplicate examples in df_warao_aribu_guide: {len(duplicates_warao_aribu)}")
print(f"Number of duplicate examples in df_ps_jw_website: {len(duplicates_ps_jw)}")
print(f"Number of duplicate examples in df_trab_warao_completo: {len(duplicates_trab)}")
print(f"Number of duplicate examples in df_bible_verses: {len(duplicates_bible)}")

In [ ]:
df_all = pd.concat([df_curso_basico_expanded, df_ps_jw_website, df_warao_aribu_guide, df_trab_warao_completo[['spanish_sentence', 'warao_sentence']], df_bible_verses])
# leave only spanish_sentence and warao_sentence columns
df_all = df_all[['warao_sentence', 'spanish_sentence']]
print(f"Total number of examples: {len(df_all)}")


In [ ]:
# extract rows that are exact duplicates
duplicates = df_all[df_all.duplicated(keep=False)]



print(f"Total number of duplicate examples: {len(duplicates)}")

display(duplicates.head(20))

In [ ]:
df_all.drop_duplicates(inplace=True)
print(f"Total number of examples after dropping duplicates: {len(df_all)}")


In [ ]:
output_filename = "parallel_data_all.csv"
output_path = os.path.join("output", output_filename)
df_all.to_csv(output_path, index=False)